In [8]:
'''
 Yacine Mahdid 2019-11-02
 This script was created for the analysis of the graph features for consciousness decoding
 The goal here is to take the CSV created in the preprocess.m file and to manipulate it to get labels, participant id and such.
 This script was tested with atruncated version of the full dataset (only alpha, only ec1 and only 1 null network) while the real dataset was getting generated

'''

'\n Yacine Mahdid 2019-11-02\n This script was created for the analysis of the graph features for consciousness decoding\n The goal here is to take the CSV created in the preprocess.m file and to manipulate it to get labels, participant id and such.\n This script was tested with atruncated version of the full dataset (only alpha, only ec1 and only 1 null network) while the real dataset was getting generated\n\n'

In [10]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

# First step will be to import the feature data and the labels
data_path = "C:/Users/biapt/Documents/only_wei_data.csv";
data = pd.read_csv(data_path)
# here what everything means:
'''
frequencies = {"alpha","beta","delta","theta","gamma"};
epochs = {'ec1','emf5','ec1','if5','emf5','eml5','ec3','ec8'};
graph = {"aec","pli"}
The ids we see are mapping to these
'''
data

,p_id,frequency,epoch,graph,window,clust_coeff_ 1,clust_coeff_ 2,clust_coeff_ 3,clust_coeff_ 4,clust_coeff_ 5,...,clust_coeff_ 77,clust_coeff_ 78,clust_coeff_ 79,clust_coeff_ 80,clust_coeff_ 81,clust_coeff_ 82,norm_avg_clust_coeff,norm_g_eff,community,small_worldness
0,1,1,1,0,1,0.14911-0.034638i,0.13497-0.044889i,0.13342-0.030166i,0.077296-0.042127i,0.1358-0.03246i,...,0.058245-0.050709i,0.090102-0.043494i,0.060118-0.044886i,0.095209-0.04444i,0.098791-0.051964i,0.0027454-0.073558i,0.98468-0.0089424i,0.95321-0.0014426i,0.145520,0.91335-0.013888i
1,1,1,1,1,1,0.11582,0.12107,0.084378,0.11873,0.13598,...,0.10672,0.12815,0.10475,0.10594,0.094395,0.10947,0.99429,0.9777,0.119570,0.96133
2,1,1,1,0,2,0.14703-0.0090889i,0.14996-0.016315i,0.16571-0.014618i,0.1596-0.015352i,0.093145-0.034014i,...,0.18248-0.017037i,0.17033-0.018049i,0.081972-0.022608i,0.15962-0.015971i,0.1065-0.025627i,0.099314-0.026233i,0.9917-0.0061787i,0.98205,0.081281,0.96258-0.0059973i
3,1,1,1,1,2,0.10341,0.10059,0.090223,0.10782,0.12332,...,0.091389,0.10349,0.11329,0.10215,0.087017,0.12059,0.99357,0.98526,0.121960,0.96887
4,1,1,1,0,3,0.081217-0.029993i,0.14106-0.026985i,0.11282-0.033511i,0.090638-0.042198i,0.046195-0.052713i,...,0.075642-0.042802i,0.086189-0.028226i,0.066865-0.029505i,0.1317-0.027234i,0.11863-0.026006i,0.12899-0.026532i,0.98701-0.010719i,0.97314-0.0031621i,0.147540,0.9499-0.023209i
5,1,1,1,1,3,0.077314,0.06537,0.07502,0.088192,0.098892,...,0.10149,0.08703,0.086551,0.071742,0.069271,0.094503,0.99139,0.98525,0.106870,0.96532
6,1,1,1,0,4,0.071576-0.039772i,0.11125-0.040812i,0.11202-0.039204i,0.11564-0.045559i,0.059102-0.044023i,...,-0.025287-0.098578i,0.035162-0.061324i,0.068133-0.030051i,0.11828-0.029599i,0.085991-0.030478i,0.081372-0.045461i,0.98355-0.017828i,0.97263-0.0034144i,0.144520,0.94677-0.02781i
7,1,1,1,1,4,0.064955,0.070198,0.061392,0.072722,0.087972,...,0.10802,0.11339,0.078162,0.068628,0.074071,0.096014,0.995,0.98494,0.113030,0.97557
8,1,1,1,0,5,0.06776-0.043365i,0.095086-0.037818i,0.069202-0.043873i,0.10044-0.052509i,0.045241-0.040734i,...,0.017788-0.065753i,0.064704-0.043857i,0.061641-0.042947i,0.079414-0.03677i,0.063235-0.034802i,0.039756-0.043991i,0.97899-0.046521i,0.97499-0.001449i,0.135510,0.93722-0.042112i
9,1,1,1,1,5,0.095983,0.087244,0.07658,0.094235,0.11312,...,0.085813,0.12285,0.074097,0.087794,0.088316,0.095469,0.99476,0.98706,0.120510,0.97609


In [12]:
# We will keep the same structure than we did in the AEC vs wPLI project for the machine learning

# Machine Learning 
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# Sklearn utils
from sklearn.base import clone
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix

# aec = 0 pli = 1
analysis_type = 1
epochs_to_keep = [1,3]
analysis = data['graph'] == analysis_type
baseline = data['epoch'] == epochs_to_keep[0]
other = data['epoch'] == epochs_to_keep[1]
subset_data = data[analysis & (baseline | other)]

clfs = [LinearDiscriminantAnalysis(solver='svd'), SVC(kernel='linear', C=0.1),SVC(kernel='linear', C=0.5), SVC(kernel='linear', C=1.0), SVC(kernel='rbf', C=0.1), SVC(kernel='rbf',C=1.0)]
selected_clf = clfs[3]

def classify(dataset, original_clf):
    clf = clone(original_clf)
    # Initialize the Result data structures
    cms = []
    accuracies = []
    reports = []
    # TODO: Check in the MATLAB file how to not have to do this + 1
    for test_id in range(1,10):
        print("Participant: " + str(test_id) + " in hold-out set:")
        
        # Split the data in a leave one subject out manner
        
        # Get the training and test dataset
        training_dataset = dataset[dataset['p_id'] != test_id]
        test_dataset = dataset[dataset['p_id'] == test_id]
        
        # Get the X and Y 
        X_train = training_dataset.iloc[:,5:]
        y_train = training_dataset['epoch']
        
        # Training the min max normalizer
        min_max_scaler = MinMaxScaler()
        min_max_scaler.fit(X_train)
        X_train = min_max_scaler.transform(X_train)
        
        X_test = test_dataset.iloc[:,5:]
        y_test = test_dataset['epoch']
        
        # Normalize the test set using a training scaler
        X_test = min_max_scaler.transform(X_test)
        
        # Fitting our model
        clf.fit(X_train, y_train)

        # predicting
        y_pred = clf.predict(X_test)
        cm = confusion_matrix(y_test, y_pred)

        report = classification_report(y_test, y_pred, output_dict=True)
        accuracy = accuracy_score(y_test, y_pred)
        print("Generalization accuracy: " + str(accuracy))
        print(cm)
        print(report)
        
        # Saving the results
        cms.append(cm)
        accuracies.append(accuracy)
        reports.append(report)
        
    return (cms,accuracies,reports)

(cms, accuracies, report) = classify(subset_data, selected_clf)
print("Mean accuracy is: " + str(np.mean(accuracies)))

Participant: 1 in hold-out set:
Generalization accuracy: 0.591304347826087
[[47 10]
 [37 21]]
{'1': {'precision': 0.5595238095238095, 'recall': 0.8245614035087719, 'f1-score': 0.6666666666666666, 'support': 57}, '3': {'precision': 0.6774193548387096, 'recall': 0.3620689655172414, 'f1-score': 0.47191011235955055, 'support': 58}, 'accuracy': 0.591304347826087, 'macro avg': {'precision': 0.6184715821812596, 'recall': 0.5933151845130067, 'f1-score': 0.5692883895131086, 'support': 115}, 'weighted avg': {'precision': 0.6189841715087157, 'recall': 0.591304347826087, 'f1-score': 0.5684416218856865, 'support': 115}}
Participant: 2 in hold-out set:
Generalization accuracy: 0.7652173913043478
[[34 23]
 [ 4 54]]
{'1': {'precision': 0.8947368421052632, 'recall': 0.5964912280701754, 'f1-score': 0.7157894736842105, 'support': 57}, '3': {'precision': 0.7012987012987013, 'recall': 0.9310344827586207, 'f1-score': 0.8, 'support': 58}, 'accuracy': 0.7652173913043478, 'macro avg': {'precision': 0.798017771